# Reconstrution Network

In [1]:
# autoreload after code has changed
%load_ext autoreload
%autoreload 2

In [62]:
import os

# move into the correct dirrectory, e.g. move up one directory level iif this cell is run for the first time
try:
    a = first_time
except NameError:
    print("Running first time, moving up one dir level")
    os.chdir('..')  # Move up one directory level to the root directory of project
    first_time = False

print("This path should be the root directory of the project: ", os.getcwd())
    


This path should be the root directory of the project:  /home/daniel/Documents/TUM/ADLR/tum-adlr-02


#### Import Libraries

In [63]:
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils
import datetime

import time



#### Project Imports

In [64]:
from data.model_classes import Mug, Bottle
from data.dataconverter import DataConverter
from data.reconstruction_dataset import *
from models.unet import UNetSmall, UNet1, count_parameters


### Dataset
Creating the dataset object and applzing transformations to the data.

In [65]:
# Outcommented because there are currently someproblems running it in the notebook

# generate data
dataconverter = DataConverter(
    classes=[Mug(),Bottle()],
    min_order = 1,
    tact_order = 2,
    tact_number=1
    
)
# set regenerate to true, if you run this after changes in dataconverter have been made
dataconverter.generate_2d_dataset(show_results=False, regenerate=False)


class mug already downloaedd. Skipping download.
class bottle already downloaedd. Skipping download.
Converting  214  files ...
Converting  498  files ...
Generating annotation CSV files for training and testing with a split ratio of 0.9:0.1.
Finished generating annotation CSV files for 7120 different shapes.


In [66]:
csv_file = './datasets/2D_shapes/annotations.csv'
root_dir = './datasets/2D_shapes'
composed = transforms.Compose([RandomOrientation(),
                               ToTensor()])

dataset = ReconstructionDataset(csv_file=csv_file,
                                root_dir=root_dir,
                                transform=composed)

Examplary data pairs:

In [67]:
example_data = dataset[58]

plt.figure()
print(example_data['image'])
#show_datapair(example_data['image'], example_data['label'])
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './datasets/2D_shapes/./datasets/2D_shapes/mug/1dd8290a154f4b1534a8988fdcee4fde/tactile_points/o3n1tactile.npy'

In [3]:
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils
import datetime

import time



#### Split Dataset into train, validation and test dataset

In [44]:
# Define the sizes for train, validation, and test
train_size = int(0.75 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


### Hyperparamters

In [41]:
num_epochs = 50
batch_size = 32
learning_rate = 1e-4
ModelClass = UNet1

def weight_init(layer):
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.ConvTranspose2d):
        init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
        if layer.bias is not None:
            init.constant_(layer.bias, 0)

    elif isinstance(layer, nn.Linear):
        init.xavier_normal_(layer.weight)
        if m.bias is not None:
            init.constant_(layer.bias, 0)

### Dataloader
Creating a dataloader based on the dataset for batch processing.

TODO: Remove spacing in figure.

In [42]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0)



batch = next(iter(train_loader))
print(batch['image'].shape)

if False:
    show_datapair_batch(next(iter(train_loader)))


FileNotFoundError: [Errno 2] No such file or directory: './datasets/2D_shapes/bottle/fda8d8820e4d166bd7134844380eaeb0/tactile_points/o8n2tactile.npy'

### Initialize Network

In [ ]:
# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ModelClass().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# initialize parameters
model.apply(weight_init)
fresh_initialized = True


print("Device: ", device)

count_parameters(model)

### Training

In [ ]:


if not fresh_initialized:
    raise ValueError("You are using model that already has been trained")
fresh_initialized = False


# Set up TensorBoard
writer = SummaryWriter(f'runs/U-Net_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')


# Training loop
log_train_period = 100
log_val_period = min(1000, len(train_loader)-1)

# used to meassure proportion of time for calculation for val loss
train_period_stime_val = time.time()
# used to meassure proportion of time for adding images to writer
train_period_stime_train = time.time()

data_loading_time=0

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    for i, batch in enumerate(train_loader):
        # todo: output should be segmentation map, read an article about it
        
        # measure time to load data 
        st_data_load = time.time()
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)
        data_loading_time += time.time() - st_data_load
        
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

        # log data, except validation loss 
        if (i+epoch) % log_train_period == log_train_period-1:  # Log train loss and images every log_train_period batches


            train_duration = time.time() - train_period_stime_train
            
            log_starttime = time.time()
            
        
            # log training loss
            writer.add_scalar('Loss/train', running_train_loss / log_train_period, epoch * len(train_loader) + i)
            

            # Log one reconstructed training image sample
            img_grid = vutils.make_grid([inputs[-1], labels[-1], outputs[-1]])
            writer.add_image('reconstructed_training_images', img_grid, global_step=epoch * len(train_loader) + i)


            # Log one reconstructed validation image sample
            batch = next(iter(train_loader))
            inputs = batch['image'].to(device)
            labels = batch['label'].to(device)
            outputs = model(inputs)
            
            img_grid = vutils.make_grid([inputs[-1], labels[-1], outputs[-1]])
            writer.add_image('reconstructed_validation_images', img_grid, global_step=epoch * len(train_loader) + i)

            # ploting images like this (they are also plotted on tensorboard, but here they are more uebersichtlich somehow)
            print("Example Rectonsturtion vrom Val Set: ")
            plt.figure()
            show_datatripple(inputs[-1].cpu().detach().numpy(), labels[-1].cpu().detach().numpy(), outputs[-1].cpu().detach().numpy())
            plt.show()


            log_duration = time.time() - log_starttime
            log_prop = log_duration / train_duration

            data_loading_prop = data_loading_time /(train_duration - data_loading_time)
            
            data_loading_time = 0
            train_period_stime_train = time.time()
            
            
            print(f'Epoch [{epoch+1 }/{num_epochs}], Step [{i+1}/{len(train_loader)}], Train Loss: {running_train_loss/log_train_period:.4f}',
                 f', Logging Time Proportion: {log_prop:.4f}, Data Loading Time Proportion: {data_loading_prop:.4f}')
            running_train_loss = 0.0

        
        # calculate and log validation loss  
        if (i+epoch) % log_val_period == log_val_period-1:  # Log val loss every log_val_period batches

            train_duration = time.time() - train_period_stime_val
            
            val_starttime = time.time()
            # calculate validation loss
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for batch in val_loader:
                    inputs = batch['image'].to(device)
                    labels = batch['label'].to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
        
            val_loss /= len(val_loader)

            # log validation loss
            writer.add_scalar('Loss/val', val_loss, epoch * len(train_loader) + i)

    
            model.train()
            
            val_duration = time.time() - val_starttime        
            val_t_prop = val_duration / train_duration
    
            
            print('##########################################################################################')        
            print(f'Epoch [{epoch+1 }/{num_epochs}], Step [{i+1}/{len(train_loader)}], Val Loss: {val_loss:.4f}', 
                    f'Val Log Time Proportion: {val_t_prop:.4f}')
            print('##########################################################################################')        

            train_period_stime_val = time.time()
            
            
            
            


# Validation step can be added here similarly and log validation loss

writer.close()